# 2022-10-24 • N-to-1 with lognormal inputs (for real now)

## Imports

In [1]:
#

In [2]:
using MyToolbox

In [3]:
@time @time_imports using VoltoMapSim;  # after precompile & w/o touching src files

   1695.9 ms  Sciplotlib 46.99% compilation time
      0.5 ms  Adapt
     70.0 ms  StructArrays
     99.7 ms  Latexify 47.64% compilation time (49% recompilation)
      1.2 ms  SpikeLab
     33.4 ms  FiniteDiff 41.20% compilation time (100% recompilation)
    334.4 ms  NLSolversBase
      4.3 ms  LsqFit
     67.8 ms  VoltoMapSim 43.76% compilation time
  2.805117 seconds (3.63 M allocations: 204.345 MiB, 14.93% gc time, 38.17% compilation time: 3% of which was recompilation)


In [3]:
@time @time_imports using VoltoMapSim;  # with precompile on

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
   1261.6 ms  Sciplotlib 60.76% compilation time
      1.1 ms  Adapt
     69.0 ms  StructArrays
    115.6 ms  Latexify 43.69% compilation time (52% recompilation)
      0.9 ms  SpikeLab
     27.8 ms  FiniteDiff 39.04% compilation time (100% recompilation)
    375.5 ms  NLSolversBase
      4.9 ms  LsqFit
     58.5 ms  VoltoMapSim 43.12% compilation time
  8.978815 seconds (3.77 M allocations: 211.816 MiB, 0.75% gc time, 13.11% compilation time: 3% of which was recompilation)


In [3]:
@time @time_imports using VoltoMapSim;  # no showprogress, match

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
   1285.2 ms  Sciplotlib 60.61% compilation time
      0.5 ms  Adapt
     77.5 ms  StructArrays
     90.0 ms  Latexify 45.25% compilation time (50% recompilation)
      1.4 ms  SpikeLab
     27.3 ms  FiniteDiff 39.55% compilation time (100% recompilation)
    354.4 ms  NLSolversBase
      5.0 ms  LsqFit
     60.8 ms  VoltoMapSim 44.61% compilation time
  9.130961 seconds (3.76 M allocations: 211.648 MiB, 0.67% gc time, 12.65% compilation time: 3% of which was recompilation)


In [3]:
@time @time_imports using VoltoMapSim;  # no showprogress, match

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
[ Info: Skipping precompilation since __precompile__(false). Importing VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e].
   1255.8 ms  Sciplotlib 61.62% compilation time
      0.5 ms  Adapt
     57.9 ms  StructArrays
     91.2 ms  Latexify 43.99% compilation time (52% recompilation)
      0.9 ms  SpikeLab
     27.4 ms  FiniteDiff 40.00% compilation time (100% recompilation)
    336.6 ms  NLSolversBase
      4.6 ms  LsqFit
  7.445982 seconds (5.97 M allocations: 328.369 MiB, 4.30% gc time, 42.12% compilation time: 8% of which was recompilation)


In [3]:
@time @time_imports using VoltoMapSim;  # after vtms cleanup

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
[ Info: Skipping precompilation since __precompile__(false). Importing VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e].
   1355.2 ms  Sciplotlib 59.54% compilation time
      0.7 ms  Adapt
     61.8 ms  StructArrays
    106.8 ms  Latexify 42.77% compilation time (52% recompilation)
      1.2 ms  SpikeLab
     36.3 ms  FiniteDiff 38.12% compilation time (100% recompilation)
    348.8 ms  NLSolversBase
      5.8 ms  LsqFit
 10.440128 seconds (10.30 M allocations: 566.882 MiB, 5.24% gc time, 55.67% compilation time: 4% of which was recompilation)


In [3]:
using ProfileView, SnoopCompile

In [4]:
tinf = @snoopi_deep (using VoltoMapSim);

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
[ Info: Skipping precompilation since __precompile__(false). Importing VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e].


In [5]:
ProfileView.view(flamegraph(tinf), windowname="snoopcompile");

## Differential equations

In [12]:
izh = @eqs begin
    
    dv/dt = (k*(v-vₗ)*(v-vₜ) - u - I_syn) / C
    du/dt = a*(b*(v-vᵣ) - u)

    I_syn = gₑ*(v-Eₑ) + gᵢ*(v-Eᵢ)

    dgₑ/dt = -gₑ / τ  # Represents sum over all exc synapses
    dgᵢ/dt = -gᵢ / τ
end

SpikeLab.Model_
 with variables {v, u, I_syn, gₑ, gᵢ}
 and parameters {C, Eᵢ, Eₑ, a, b, k, vᵣ, vₗ, vₜ, τ}


## Parameters

In [27]:
params = (
    # Cortical regular spiking (same as always)
    C  =  100    * pF,
    k  =    0.7  * (nS/mV),
    vₗ = - 60    * mV,
    vₜ = - 40    * mV,
    a  =    0.03 / ms,
    b  = -  2    * nS,
    # Not in model eqs above (yet)
    vₛ =   35    * mV,  # spike
    vᵣ = - 50    * mV,  # reset
    Δu =  100    * pA,

    # Synapses
    Eₑ =   0 * mV,
    Eᵢ = -80 * mV,  # Higher than Nto1 (was -65); same as nets.
    τ  =   7 * ms,
);

## Init buffers

In [30]:
init = (
    v  = params.vᵣ,
    u  = 0 * pA,
    gₑ = 0 * nS,
    gᵢ = 0 * nS,
    I_syn = 0 * nA,
)
vars = CVec{Float64}(init)
diff = similar(vars)
diff .= 0

ComponentVector{Float64}(v = 0, u = 0, gₑ = 0, gᵢ = 0, I_syn = 0)

In [ ]:
init.gᵢ += w

In [21]:
showsome(poisson_spikes(4Hz, 10minutes))

2381-element Vector{Float64}:
    1: 0.206
    2: 0.301
    ⋮
 1391: 352
    ⋮
 1771: 449
    ⋮
 2380: 600
 2381: 600


In [31]:
izh.generated_func

:((diff, vars, params)->begin
          @unpack (v, u, I_syn, gₑ, gᵢ) = vars
          @unpack (C, Eᵢ, Eₑ, a, b, k, vᵣ, vₗ, vₜ, τ) = params
          diff.v = ((k * (v - vₗ) * (v - vₜ) - u) - I_syn) / C
          diff.u = a * (b * (v - vᵣ) - u)
          vars.I_syn = gₑ * (v - Eₑ) + gᵢ * (v - Eᵢ)
          diff.gₑ = -gₑ / τ
          diff.gᵢ = -gᵢ / τ
      end)

In [29]:
# izh.f(diff, vars, params)